**Setup**

In [1]:
import pandas as pd
import numpy as np

def agg_mean_pm_se(x):
    """Return 'avg ± se' for a 1-D array-like."""
    mean = x.mean()
    se   = x.std(ddof=1) / np.sqrt(len(x))
    return f"{mean:.2e} ± {se:.2e}"

def sort_categorical(df, column, ordered_categories):
    return (
        df.assign(**{ column: lambda df: df[column].astype("category").cat.reorder_categories(ordered_categories) })
        .sort_values(column)
    )

df = (
    pd.read_csv("results.csv")
    .rename(columns={
        "layer_kind": "Layer",
        "layer_training_mode": "Training mode",
        "layer_init_kind": "Initialization",
        "transformation": "Transformation",
        "horizontal_displacement": "Horizontal displacement",
        "vertical_displacement": "Vertical displacement",
        "error": "Error",
    })
    .replace({
        "Layer": {
            "BatchNorm2d": "BatchNorm",
            "InstanceNorm2d": "InstanceNorm",
            "LayerNorm": "LayerNorm-CHW",
            "LayerNorm_AF": "LayerNorm-AF",
            "LayerNorm_C": "LayerNorm-C"
        },
        "Transformation": {
            "shift": "Shifts",
            "translate": "Translations"
        },
        "Training mode": {
            True: "Train",
            False: "Eval"
        },
        "Initialization": {
            "default": "Default",
            "normal": "Gaussian"
        }
    })
    .pipe(lambda df: sort_categorical(df, "Layer", ["BatchNorm", "InstanceNorm", "LayerNorm-CHW", "LayerNorm-C", "LayerNorm-AF"]))
    .pipe(lambda df: sort_categorical(df, "Transformation", ["Shifts", "Translations"]))
    .pipe(lambda df: sort_categorical(df, "Training mode", ["Train", "Eval"]))
)

**Equivariance error of normalization layers.**

In [2]:
df_tbl = (
    df
    .pivot_table(index="Layer", columns="Transformation", values="Error", aggfunc=agg_mean_pm_se)
    .pipe(lambda df: df.rename_axis(columns=None))
)
display(df_tbl)

/tmp/ipykernel_31796/650136440.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  .pivot_table(index="Layer", columns="Transformation", values="Error", aggfunc=agg_mean_pm_se)


,Shifts,Translations
Layer,,
BatchNorm,9.23e-09 ± 2.87e-12,1.28e-06 ± 1.70e-09
InstanceNorm,9.58e-09 ± 5.28e-12,7.15e-06 ± 1.89e-08
LayerNorm-CHW,4.97e-01 ± 3.54e-04,4.97e-01 ± 3.53e-04
LayerNorm-C,4.65e-09 ± 7.06e-12,2.44e-03 ± 2.66e-06
LayerNorm-AF,9.17e-09 ± 4.07e-12,8.04e-07 ± 2.59e-09


**Equivariance error for the two modes of batch normalization.**

In [3]:
df_tbl = (
    df
    .pipe(lambda df: df[df["Layer"] == "BatchNorm"])
    .pivot_table(index="Training mode", columns="Transformation", values="Error", aggfunc=agg_mean_pm_se)
    .pipe(lambda df: df.rename_axis(columns=None))
)
display(df_tbl)

/tmp/ipykernel_31796/1061155862.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  .pivot_table(index="Training mode", columns="Transformation", values="Error", aggfunc=agg_mean_pm_se)


,Shifts,Translations
Training mode,,
Train,9.51e-09 ± 3.80e-12,2.55e-06 ± 3.16e-09
Eval,8.95e-09 ± 4.29e-12,9.67e-09 ± 3.44e-12
